In [3]:
import numpy as np
box = [0,1,0,1]                  #方程所在的区域
ny = 8
nx = ny 
NN = (nx+1)*(ny+1)               #节点数
NC = nx*ny                       #边数
node = np.zeros((NN,2))          #生成一个node零矩阵
#complex() 函数用于创建一个值为 real + imag * j 的复数或者转化一个字符串或数为复数
#mgrid()函数生成在0到1均匀八等份的节点
Y,X= np.mgrid[box[0]:box[1]:complex(nx+1),box[2]:box[3]:complex(ny+1)]
node[:,0] = X.flatten()
node[:,1] = Y.flatten()
cell = np.zeros((2*NC,3),dtype = np.int)
sign = np.arange(NN).reshape(ny+1,nx+1)
cell[:NC,0] = sign[0:-1,1:].flatten() 
cell[:NC,1] = sign[1:,1:].flatten()
cell[:NC,2] = sign[0:-1,0:-1].flatten()
cell[NC:,0] = sign[1:,0:-1].flatten() 
cell[NC:,1] = sign[0:-1,0:-1].flatten()
cell[NC:,2] = sign[1:,1:].flatten()
v0 = node[cell[:, 2], :] - node[cell[:, 1], :]
v1 = node[cell[:, 0], :] - node[cell[:, 2], :]
v2 = node[cell[:, 1], :] - node[cell[:, 0], :]
length = np.cross(v2, -v1)
Dlambda = np.zeros((2*NC, 3, 2), dtype=np.float)
W = np.array([[0, 1], [-1, 0]], dtype=np.int)
Dlambda[:,0,:] = v0@W/length.reshape(-1,1)
Dlambda[:,1,:] = v1@W/length.reshape(-1,1)
Dlambda[:,2,:] = v2@W/length.reshape(-1,1)
A1=np.zeros((2*NC,3,3))
A=np.zeros((NN,NN))
for i in np.arange(2*NC):
    for j in np.arange(3):
        for k in np.arange(3):
            A1[i,j,k] = Dlambda[i][j]@Dlambda[i][k]*length[i]/2
for i in np.arange(2*NC):
    for j in np.arange(3):
        for k in np.arange(3):
            A[cell[i][j],cell[i][k]]=A[cell[i][j],cell[i][k]]+A1[i][j][k]            
def f(x,y):
    f = -2*(x*(x-1)+y*(y-1))
    return f
b=np.zeros(NN)
B=np.zeros((2*NC,3))
for i in np.arange(2*NC):
    for j in np.arange(3):
        B[i,j]=f(node[cell[:, j], :][i][0],node[cell[:, j], :][i][1])*length[i]/6
for i in np.arange(2*NC):
    for j in np.arange(3):
        b[cell[i][j]]=b[cell[i][j]]+B[i][j]
def u(x,y):
    u = x*(x-1)*y*(y-1)
    return u
point_bound_down = np.arange(nx+1)
point_bound_up = np.arange(ny*(nx+1),(ny+1)*(nx+1))
point_bound_left = np.arange((nx+1),(ny)*(nx+1),(nx+1))
point_bound_right = np.arange((2*nx+1),(ny+1)*(nx+1)-1,(nx+1))
point_bound_1 = np.append(point_bound_down,point_bound_up)
point_bound_2 = np.append(point_bound_left,point_bound_right)
point_bound = np.append(point_bound_1,point_bound_2)
point_bound = sorted(point_bound)
point_inside = np.setdiff1d(range(NN),point_bound)
u_bound=[u(node[i][0],node[i][1])for i in point_bound]
b-=A[:,point_bound]@u_bound
b[point_bound]=u_bound
A[point_bound,:]=0
A[:,point_bound]=0
A[point_bound,point_bound]=1
uh = np.linalg.solve(A,b)
uu = [u(node[i][0],node[i][1])for i in range(NN)]
# A_inside = A[point_inside,:][:,point_inside]
# b_inside = b[point_inside]
# uh_inside = uh[point_inside]
u_h = np.zeros((2*NC,3))
for i in np.arange(2*NC):
    u_h[i,:] = uh[cell[i,:]]
u_u = np.zeros((2*NC,3))
for i in np.arange(2*NC):
    for j in np.arange(3):
        u_u[i,j] = u(node[cell[:,j],:][i][0],node[cell[:,j],:][i][1])
for i in np.arange(2*NC):
    e = ((u_u-u_h)**2*length[i]/6).sum()
    L2error = np.sqrt(e)
print(L2error)

5.09920925636e-18


In [4]:
uu-uh

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.46944695e-18,   3.46944695e-18,
         0.00000000e+00,   1.73472348e-18,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         6.93889390e-18,   6.93889390e-18,   6.93889390e-18,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.46944695e-18,   0.00000000e+00,
         1.38777878e-17,   6.93889390e-18,   0.00000000e+00,
         0.00000000e+00,   3.46944695e-18,   0.00000000e+00,
         0.00000000e+00,   3.46944695e-18,   6.93889390e-18,
         6.93889390e-18,   0.00000000e+00,   0.00000000e+00,
        -6.93889390e-18,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,